In [ ]:
%pip install transformers peft torch

# 1. Load the LLaMA model and tokenizer:

from transformers import LlamaTokenizer, LlamaForSequenceClassification
from peft import PeftModel, PeftConfig

# Load pre-trained LLaMA tokenizer and model
tokenizer = LlamaTokenizer.from_pretrained('meta-llama/Llama-3-8b')
model = LlamaForSequenceClassification.from_pretrained('meta-llama/Llama-3-8b')

# 1. Prepare your dataset:

texts = ["I love programming.", 
"Python is great!",
 "I enjoy learning new things."]
labels = [1, 1, 1]  # Example labels

inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")

# 1. Define a simple dataset class:

import torch

class SimpleDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

dataset = SimpleDataset(inputs, labels)

# 1. Set up P-tuning configuration and training:

from transformers import Trainer, TrainingArguments

# Define P-tuning configuration
peft_config = PeftConfig(
model=model,
tokenizer=tokenizer,
task_type="sequence_classification",
prompt_length=10,  # Length of the prompt
num_virtual_tokens=20  # Number of virtual tokens
)

# Wrap the model with PeftModel
peft_model = PeftModel(peft_config)

# Define training arguments
training_args = TrainingArguments(
output_dir='./results',
num_train_epochs=3,
per_device_train_batch_size=2,
warmup_steps=10,
weight_decay=0.01,
logging_dir='./logs',
)

# Initialize Trainer
trainer = Trainer(
model=peft_model,
args=training_args,
train_dataset=dataset,
)

# Train the model
trainer.train()